In [ ]:
# https://milvus.io/docs/hybrid_search_with_milvus.md
# BGE-M3: https://huggingface.co/BAAI/bge-m3

# https://milvus.io/docs/chinese-analyzer.md 中文用的jieba
# jieba库好多年没有更新过了 而且对英文的支持也不好

下面是 本 session 对话内容的结构化总结笔记（中文、清晰、有组织，可直接放进你的项目文档里）。

⸻

📝 Milvus + BGE-M3 + 中文/英文全文检索体系：对话总结笔记

1. 用户问题起点：

“Milvus 如何做中文全文检索？有没有多语言 analyzer？jieba 可处理中英混合吗？是否还需要 Elasticsearch？”

⸻

2. Milvus 的全文检索能力本质

Milvus 的 “full-text search” 能力 不是传统搜索引擎级别：
	•	❌ 没有 tokenizer pipeline
	•	❌ 没有 BM25
	•	❌ 没有多语言 analyzer
	•	❌ 不支持 ik / jieba / smartcn / Lucene analyzers
	•	✔ 只支持简单的 substring 匹配
	•	✔ 支持向量搜索（dense search）
	•	✔ 现在新增 稀疏向量（sparse vector）索引，可模拟 lexical 检索

👉 因此：Milvus 的全文检索 本质是向量检索 + 稀疏倒排索引（由模型生成），不是依赖分词器。

⸻

3. jieba 的现状与能力
	•	✔ jieba 能做中文分词
	•	✔ 英文会作为一个 token 处理
	•	✔ 中英文混合能处理但很粗糙
	•	❌ 不做英文的 NLP tokenization（词形还原、复数等）
	•	❌ 多年不更新（处于停滞状态）
	•	❌ 不适合严肃全文检索
	•	❌ 不适合现代 RAG / hybrid retrieval 系统

结论：jieba 可以作为小工具，不能作为核心全文检索方案。

⸻

4. 什么是 BGE-M3？（对话中的重点）

BGE-M3 是 北京智源发布的全球最强的通用 embedding 模型之一，特点：

M3 = Multi-Function + Multi-Lingual + Multi-Granularity

它同时输出：
	•	dense embedding（语义向量） → Milvus dense search
	•	sparse embedding（稀疏向量） → Milvus sparse inverted index（相当于 BM25-like）

优势：
	•	✔ 原生支持中英文混合
	•	✔ 支持多语言
	•	✔ 支持 lexical + semantic
	•	✔ 用 sparse embedding 替代传统分词器（不再需要 jieba / ik）
	•	✔ 非常适合 RAG、教育内容、智库文档、多语言技术文本

总结：BGE-M3 = 分词器 + BM25 替代 + 语义模型 + 多语言模型。

⸻

5. 关键突破点：Milvus 官方已经原生支持 BGE-M3（dense + sparse + hybrid）

你发现的 Milvus 文档说明：

Milvus 现在直接支持 dense search + sparse search + hybrid search，且官方提供了 BGEM3EmbeddingFunction。

流程：
	1.	文本 → BGE-M3
	•	dense vector
	•	sparse vector
	2.	Milvus 建 collection：
	•	FLOAT_VECTOR
	•	SPARSE_FLOAT_VECTOR（新能力）
	3.	查询时：
	•	dense 搜索（语义）
	•	sparse 搜索（关键词）
	•	hybrid 搜索（得分加权融合）

效果 = BM25 + 向量语义 的组合搜索，但无需 Elasticsearch。

👉 Milvus 现在自身就能做 hybrid search，因此不一定非要 ES 了。

⸻

6. 是否还需要 Elasticsearch？

✔ 可以不需要 ES 的场景（你当前就是）
	•	RAG 检索
	•	文本 QA
	•	教育/论文/智库资料检索
	•	中英文混合文本
	•	重点是召回与语义，不是复杂 DSL

Milvus + BGE-M3 足够优秀，架构简单，还更现代。

⸻

❗ 仍然需要 ES 的情况

如果你未来要：
	•	复杂过滤：must / must_not / filter
	•	多层嵌套 bool 查询
	•	聚合统计（按年份、机构、课题分类等）
	•	正则/通配符/pinyin 搜索
	•	非文本字段排序（时间权重 + 热度）
	•	大规模 BI 分析

→ ES 仍然不可替代。

现在阶段（你的智库系统 + RAG + 教材知识库）：

👉 完全可以不加 ES。

⸻

7. 最终系统结构推荐（结合你的场景）

⭐ 推荐架构（最适合你当前项目）

PDF / Word / 教材内容
      ↓
   文本解析
      ↓
    chunk
      ↓
 BGE-M3 (dense + sparse)
      ↓
 ┌────────────┐
 │   Milvus   │
 │ dense index│  ← semantic retrieval
 │ sparse idx │  ← keyword-like retrieval
 │ hybrid     │  ← dense + sparse (强大!)
 └────────────┘
      ↓
   RAG / QA

这是目前最现代、最实用、最适合中文/英文技术文本的系统架构。

⸻

🔚 本 session 的一句话总结

Milvus + BGE-M3 已经可以完成传统“全文检索 + 语义检索”的混合任务，在当前你的 RAG / 智库内容检索需求中，可以完全不使用 Elasticsearch。jieba 已经过时，BGE-M3 的 sparse embedding 已经自然替代了分词器。

⸻

如果你愿意，我还能帮你继续生成：
	•	你的 Milvus + BGE-M3 collection schema 最佳模板
	•	端到端 ingest pipeline（PDF→chunk→embedding→Milvus）
	•	hybrid_search(weight_dense, weight_sparse) 查询接口设计
	•	小型 demo 代码

叫我继续即可。

In [ ]:
##  Hybrid Search with Milvus
# BGE-M3 model can convert text into dense and sparse vectors.
# - Dense Retrieval: Utilizes semantic context to understand the meaning behind queries.
# - Sparse Retrieval: Emphasizes keyword matching to find results based on specific terms, equivalent to full-text search.
# - Hybrid Retrieval: Combines both Dense and Sparse approaches, capturing the full context and specific keywords for comprehensive search results.

In [1]:
## Load and Prepare Data
import pandas as pd

file_path = "quora_duplicate_questions.tsv"
df = pd.read_csv(file_path, sep="\t")
questions = set()
for _, row in df.iterrows():
    obj = row.to_dict()
    questions.add(obj["question1"][:512])
    questions.add(obj["question2"][:512])
    if len(questions) > 500:  # Skip this if you want to use the full dataset
        break

docs = list(questions)

# example question
print(docs[0])

How can I keep my nose from getting stuffy at night?


In [2]:
## Use BGE-M3 Model for Embeddings
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
dense_dim = ef.dim["dense"]

# Generate embeddings using BGE-M3 model
docs_embeddings = ef(docs)

/Users/zhangzhong/src/LearnMilvus/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


start to install package: datasets
successfully installed package: datasets
start to install package: peft


ERROR: Exception:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 107, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pip/_internal/cli/base_command.py", line 98, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pip/_internal/cli/req_command.py", line 85, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pip/_internal/commands/install.py", line 388, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pip/_internal/resolution/resolvelib/resolver.py", line 99, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib

ValueError: install error Command 'pip install -q peft' returned non-zero exit status 2.